### 實作目標：Fine tune一個resnet101分類40種動物

#### 實作進度：
- 10/16:
    - 將資料讀入改成flow.from_directory [參考](https://stackoverflow.com/questions/41815354/keras-flow-from-directory-over-or-undersample-a-class)
    - 配合class weights去讓資料平衡
    - 自己犯傻，弄錯preprocess input，目前已修正資料集問題，finetune成功。

- 10/15: 
    - 由於資料分佈不平均，與資料數量太少（受限於硬體設備）
    - 所以使用RandomOverSampling 與 ImageDataGenerator增加圖片數量，但效果不好，速度很慢
    - acc有穩定上升，但是val_acc只有1%，說明資料集嚴重出問題。
    - [ImageDataGenerator](https://medium.com/@shihaoticking/%E5%AF%A6%E4%BD%9C%E8%B3%87%E6%96%99%E5%BC%B7%E5%8C%96-data-augmentation-%E5%AF%A6%E7%8F%BE%E5%9C%96%E7%89%87%E7%BF%BB%E8%BD%89-%E5%B9%B3%E7%A7%BB-%E7%B8%AE%E6%94%BE-4b37d4400ffb)
    - [RandomOverSampling Combine ImageDataGenerator](https://medium.com/analytics-vidhya/how-to-apply-data-augmentation-to-deal-with-unbalanced-datasets-in-20-lines-of-code-ada8521320c9)


In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import tensorflow
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.keras import balanced_batch_generator
# from sklearn.utils import class_weight

## Import data

In [8]:
### change the dataset here###
dataset = 'AWA2'
##############################


batch_size = 16
train_dir = './data/{}/IMG/train'.format(dataset)
val_dir = './data/{}/IMG/val'.format(dataset)
IMG_SHAPE = 224
epochs = 15

if dataset == 'SUN':
    class_attr_shape = (102, )
    class_attr_dim = 102
    class_num = 717
    seen_class_num = 645
    unseen_class_num = 72
elif dataset == 'CUB':
    class_attr_shape = (312, )
    class_attr_dim = 312
    class_num = 200
    seen_class_num = 150
    unseen_class_num = 50
elif dataset == 'AWA2':
    class_attr_shape = (85, )
    class_attr_dim = 85
    class_num = 50
    seen_class_num = 40
    unseen_class_num = 10
elif dataset == 'plant':
    class_attr_shape = (46, )
    class_attr_dim = 46
    class_num = 38
    seen_class_num = 25
    unseen_class_num = 13

In [6]:
image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_data_gen = image_gen.flow_from_directory(
        batch_size=batch_size,
        directory=train_dir,
        shuffle=True,
        color_mode="rgb",
        target_size=(IMG_SHAPE,IMG_SHAPE),
        class_mode='categorical',
        seed = 42
    )

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: './data/SUN/IMG/train'

In [6]:
image_gen_val = ImageDataGenerator(preprocessing_function=preprocess_input)

val_data_gen = image_gen_val.flow_from_directory(
    batch_size=batch_size,
    directory=val_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='categorical',
    color_mode="rgb",
    seed = 42
)

Found 12895 images belonging to 645 classes.


In [7]:
# class_weights = class_weight.compute_class_weight(
#            'balanced',
#             np.unique(train_data_gen.classes), 
#             train_data_gen.classes)

## Fine tune or Retrain ResNet101

In [7]:
base_model = ResNet101(weights = 'imagenet', include_top = False)


In [8]:


# # lock the model
# for layer in base_model.layers:
#     layer.trainable = False

# add a global averge pollinf layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# add a dense
x = Dense(1024, activation='relu')(x)

# add a classifier
predictions = Dense(seen_class_num, activation='softmax')(x)

# Constructure
model = Model(inputs=base_model.input, outputs=predictions)


# compile
# model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#               , loss='categorical_crossentropy',metrics=['accuracy'])

model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
             , loss='categorical_crossentropy',metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10,verbose=1)

STEP_SIZE_TRAIN=train_data_gen.n//train_data_gen.batch_size
STEP_SIZE_VALID=val_data_gen.n//val_data_gen.batch_size

model.fit_generator(train_data_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs = epochs,
                    validation_data=val_data_gen,
                    validation_steps=STEP_SIZE_VALID,
#                     class_weight=class_weights,
                    callbacks = [early_stopping]
                  )

Epoch 1/15
 222/3215 [=>............................] - ETA: 26:12 - loss: 6.4597 - accuracy: 0.0196

KeyboardInterrupt: 

In [10]:
epochs = 10

for layer in model.layers[:335]:
    layer.trainable = False
for layer in model.layers[335:]:
    layer.trainable = True

from keras.optimizers import SGD

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss = 'categorical_crossentropy',metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10,verbose=1)



STEP_SIZE_TRAIN=train_data_gen.n//train_data_gen.batch_size
STEP_SIZE_VALID=val_data_gen.n//val_data_gen.batch_size

model.fit_generator(train_data_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs = epochs,
                    validation_data=val_data_gen,
                    validation_steps=STEP_SIZE_VALID,
#                     class_weight=class_weights,
                    callbacks = [early_stopping]
                  )

Epoch 1/10
3215/3215 [==============================] - 615s 191ms/step - loss: 4.2613e-04 - acc: 0.9999 - val_loss: 5.0152 - val_acc: 0.3598
Epoch 2/10
3215/3215 [==============================] - 611s 190ms/step - loss: 6.4808e-04 - acc: 0.9999 - val_loss: 5.0131 - val_acc: 0.3593
Epoch 3/10
3215/3215 [==============================] - 610s 190ms/step - loss: 5.9322e-04 - acc: 0.9999 - val_loss: 5.0059 - val_acc: 0.3598
Epoch 4/10
3215/3215 [==============================] - 611s 190ms/step - loss: 6.4363e-04 - acc: 0.9998 - val_loss: 5.0029 - val_acc: 0.3601
Epoch 5/10
3215/3215 [==============================] - 610s 190ms/step - loss: 5.5340e-04 - acc: 0.9999 - val_loss: 5.0115 - val_acc: 0.3597
Epoch 6/10
3215/3215 [==============================] - 611s 190ms/step - loss: 6.0175e-04 - acc: 0.9999 - val_loss: 5.0172 - val_acc: 0.3588
Epoch 7/10
3215/3215 [==============================] - 611s 190ms/step - loss: 5.0810e-04 - acc: 0.9999 - val_loss: 4.9998 - val_acc: 0.3601
Epoch 

In [11]:
new_model = Model(model.inputs, model.layers[-3].output)

new_model.summary()

new_model.save('./model/{}/FineTuneResNet101.h5'.format(dataset))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

## Evaluate

In [12]:
model.evaluate_generator(
    generator=val_data_gen,
    steps=STEP_SIZE_VALID
)

[5.008560806623898, 0.35970496894409937]